In [1]:
#include "Spielbrett.hpp"

In [2]:
SVG brett = SVG(800,800);

In [3]:
brett.drawText("Monopoly",130,-100,40);

In [4]:
int spieleranzahl = 0;
do{
    cout<<"Wie viele Spieler spielen? ";
    cin>>spieleranzahl;
}while(spieleranzahl < 1 || spieleranzahl >4);

Wie viele Spieler spielen? 2


In [5]:
Spielbrett spielbrett = Spielbrett(spieleranzahl, &brett);
list<Spieler*> spieler = spielbrett.getSpielerListe();

Text txt = Text();
string key = "";
//last key - W zum würfeln 
// neue funktion aufrufen (while-do) 

In [6]:
cout << "gebt euch namen!\n"; 
string name = "";
for(Spieler* s: spieler){
    cin >> name; 
    s->setName(name); 
}

gebt euch namen!
a
b


In [14]:
void hypothek(Spieler &s){
    
    AlgoViz::sendText("möchtest du eine Hypothek aufnehmen, auflösen oder weiter spielen? (1 / 2 / 3)"); 
    key = brett.lastKey();
    
    while(key!="3" && key != "1" && key != "2" ){
        key = brett.lastKey();
    }
    
    int num; 
    vector <Straße*> streets = s.getStreets();
    string allStr = "";
    
    if(key == "1"){
        if(streets.size() > 0){
            AlgoViz::sendText("für welche Straße möchtest du eine Hypothek aufnehmen? (0 - n)"); 
            AlgoViz::sleep(3000); 
            
            for(Straße *str: streets){
                cout << str->getName() << "\n"; 
            }
        }    
        cin >> num; 
        s.HypothekAufnehmen(*streets.at(num));
    }
    
    else if(key=="2"){
        AlgoViz::sendText("welche deiner Straßen möchtest du zurück kaufen? (0-n)"); 
        AlgoViz::sleep(3000);
        
        for(Straße *str: streets){
            if(str->getHypothek()){
                cout << str->getName() << "\n"; 
                }    
            }
        
        cin >> num; 
        s.HypothekAufloesen(*streets.at(num)); 
    }
    
}

In [16]:
void kaufen(Spieler &spieler, Straße &straße){
    
    //Funktion zum kaufen einer Straße
    
    AlgoViz::sendText("möchtest du diese Straße kaufen? 'j' / 'n'?");
    key = brett.lastKey(); 
    
    while(key != "j" && key != "n"){
        key = brett.lastKey();
    }
    
    if(key == "j"){   

        if(spieler.kaufenMoeglich(straße)){
            spieler.straßeKaufen(straße);
            AlgoViz::sendText("die Straße wurde erfolgreich gekauft!"); 
            AlgoViz::sleep(3000);
            hypothek(spieler);
            
        }
        else{
            AlgoViz::sendText("du hast nicht genug Geld um diese Straße zu kaufen");
            AlgoViz::sleep(3000);
            hypothek(spieler);
            

        }
        
        
    }
    hypothek(spieler);
}

In [17]:
bool zahlen(Spieler &s, Straße &straße){
    
    AlgoViz::sendText("du bist auf jemand anderes Straße gelandet!");
    AlgoViz::sleep(3000);
    
    //zahlen bestimmt nachher spiel und damit den Spielabbruch
    
    bool zahlen = false; 
    
    for(Spieler* s2: spieler){
        if(s2->besitzeStraße(straße)){
            //prüfe ob wir überhaupt Miete zahlen können 
                        
            if(s2->mieteZahlenMoeglich(straße)){
                //falls ja:
                
                s.mieteZahlen(*s2, straße);
                int miete = straße.getMiete();

                AlgoViz::sendText("du musstest "+ to_string(miete) +" zahlen");
                AlgoViz::sleep(3000);
            }
            
            //falls nein: 
            else{
                AlgoViz::sendText("du kannst dir die Miete leider nicht leisten - Game Over!");
                AlgoViz::sleep(5000); 
                hypothek(s);
                }
           
            } 
    }
    
    return zahlen;
}

In [18]:
bool spielzug(Spieler &spieler, Spielbrett &spielbrett){
    
    //bool um das Spiel zu beenden
    bool spiel = false;  
    
    string name = spieler.getName() + " ist am Zug";
    AlgoViz::sendText(name);   
    AlgoViz::sleep(3000);
    
    //mehr Interaktion
    AlgoViz::sendText("um zu würfeln, drücke 'w'!");
    key = brett.lastKey();
    
    while(key != "w"){
        key = brett.lastKey(); 
    }
    
    int wuerfel = spieler.wuerfeln();
    AlgoViz::sendText("du hast eine "+ to_string(wuerfel)+" gewürfelt"); 
    AlgoViz::sleep(2000);
    spielbrett.laufen(spieler, wuerfel); 
    
    Straße* str = spielbrett.getStraße(spieler.getFeldPos()); 
    string street = "du bist auf " + str->getName() + " gelandet!"; 
    AlgoViz::sendText(street);
    AlgoViz::sleep(3000);
    
    bool verkauft = str->getVerkauft();
    
    if(!verkauft){
        //nochmal einzelne Funktionen fürs kaufen und zahlen     
        kaufen(spieler, *str);
    }
    
    else if(!spieler.besitzeStraße(*str)){
        spiel = zahlen(spieler, *str);
    }
    
    return spiel;
    
}


In [ ]:
    //main programm// 

bool spielEnde = false;
int i = 0;

while(!spielEnde){
    for(Spieler* s: spieler){
        i++;
        string kapAnz = "Kapital von " + (s->getName()) + " : " + to_string(floor(s->getKapital()));
        Text txt = Text(kapAnz, 150, -50, &brett);
        spielEnde = spielzug(*s, spielbrett);
        
//         AlgoViz::sendText("um weiter zu spielen, drücke 'c'"); 
//         key = brett.lastKey();
       
    
//         while(key!="c"){
//             key = brett.lastKey();
//         }
        //ZUM TESTEN
        if(i>1){
            spielEnde = true;
        }
    }
}

//Ruft Destruktoren für Spieler und Straßen auf
// spielbrett.deleteSpielerStraßen();
// brett.clear();



